# Pagbuo Gamit ang Meta Family Models

## Panimula

Sasaklawin ng araling ito ang:

- Pag-explore sa dalawang pangunahing modelo ng Meta family - Llama 3.1 at Llama 3.2
- Pag-unawa sa mga gamit at sitwasyon para sa bawat modelo
- Halimbawa ng code para ipakita ang natatanging katangian ng bawat modelo

## Ang Meta Family ng mga Modelo

Sa araling ito, tatalakayin natin ang 2 modelo mula sa Meta family o "Llama Herd" - Llama 3.1 at Llama 3.2

Ang mga modelong ito ay may iba’t ibang variant at makukuha sa Github Model marketplace. Narito ang karagdagang detalye tungkol sa paggamit ng Github Models para [mag-prototype gamit ang AI models](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst).

Mga Variant ng Modelo:
- Llama 3.1 - 70B Instruct
- Llama 3.1 - 405B Instruct
- Llama 3.2 - 11B Vision Instruct
- Llama 3.2 - 90B Vision Instruct

*Tandaan: Ang Llama 3 ay available din sa Github Models ngunit hindi ito tatalakayin sa araling ito*


## Llama 3.1

Sa 405 Bilyong Parameter, kabilang ang Llama 3.1 sa kategorya ng open source na LLM.

Ang mode na ito ay isang pag-upgrade mula sa naunang release na Llama 3 sa pamamagitan ng pag-aalok ng:

- Mas malaking context window - 128k tokens kumpara sa 8k tokens
- Mas mataas na Max Output Tokens - 4096 kumpara sa 2048
- Mas mahusay na Multilingual Support - dahil sa pagdami ng training tokens

Dahil dito, kayang hawakan ng Llama 3.1 ang mas komplikadong mga use case kapag gumagawa ng GenAI applications kabilang ang:
- Native Function Calling - kakayahang tumawag ng mga external na tool at function sa labas ng LLM workflow
- Mas mahusay na RAG Performance - dahil sa mas mataas na context window
- Synthetic Data Generation - kakayahang gumawa ng epektibong data para sa mga gawain tulad ng fine-tuning


### Native Function Calling

Ang Llama 3.1 ay mas pinahusay para maging mas epektibo sa pagtawag ng mga function o tool. Mayroon din itong dalawang built-in na tool na kayang tukuyin ng modelo kung kailan dapat gamitin, depende sa prompt ng user. Ang mga tool na ito ay:

- **Brave Search** - Puwedeng gamitin para makakuha ng pinakabagong impormasyon gaya ng lagay ng panahon sa pamamagitan ng web search
- **Wolfram Alpha** - Puwedeng gamitin para sa mas komplikadong kalkulasyon sa matematika kaya hindi mo na kailangang gumawa ng sarili mong mga function.

Puwede ka ring gumawa ng sarili mong custom na tool na pwedeng tawagin ng LLM.

Sa code example sa ibaba:

- Ipinapakita kung paano ideklara ang mga available na tool (brave_search, wolfram_alpha) sa system prompt.
- Magpapadala ng user prompt na nagtatanong tungkol sa lagay ng panahon sa isang partikular na lungsod.
- Sasagot ang LLM gamit ang tool call papunta sa Brave Search tool na ganito ang itsura `<|python_tag|>brave_search.call(query="Stockholm weather")`

*Note: Sa example na ito, tool call lang ang ginagawa. Kung gusto mong makuha ang resulta, kailangan mong gumawa ng libreng account sa Brave API page at ideklara mismo ang function.


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import AssistantMessage, SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "meta-llama-3.1-405b-instruct"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)


tool_prompt=f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 23 July 2024

You are a helpful assistant<|eot_id|>
"""

messages = [
    SystemMessage(content=tool_prompt),
    UserMessage(content="What is the weather in Stockholm?"),

]

response = client.complete(messages=messages, model=model_name)

print(response.choices[0].message.content)





### Llama 3.2

Kahit isa itong LLM, isa sa mga limitasyon ng Llama 3.1 ay ang kakulangan sa multimodality. Ibig sabihin, hindi ito kayang tumanggap ng iba’t ibang uri ng input gaya ng mga larawan bilang prompt at magbigay ng sagot. Isa ito sa mga pangunahing tampok ng Llama 3.2. Kasama rin sa mga tampok na ito ang:

- Multimodality – may kakayahang suriin ang parehong text at image prompts
- Maliit hanggang katamtamang laki ng mga variant (11B at 90B) – nagbibigay ito ng flexible na opsyon para sa deployment,
- Mga variant na text-only (1B at 3B) – nagbibigay-daan ito para magamit ang model sa edge / mobile devices at maghatid ng mababang latency

Ang suporta sa multimodal ay malaking hakbang sa mundo ng open source na mga modelo. Sa halimbawa ng code sa ibaba, parehong larawan at text prompt ang ginagamit para makakuha ng pagsusuri ng larawan mula sa Llama 3.2 90B.

### Multimodal na Suporta gamit ang Llama 3.2


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "Llama-3.2-90B-Vision-Instruct"


client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(
            content="You are a helpful assistant that describes images in details."
        ),
        UserMessage(
            content=[
                TextContentItem(text="What's in this image?"),
                ImageContentItem(
                    image_url=ImageUrl.load(
                        image_file="sample.jpg",
                        image_format="jpg",
                        detail=ImageDetailLevel.LOW)
                ),
            ],
        ),
    ],
    model=model_name,
)

print(response.choices[0].message.content)

## Hindi dito nagtatapos ang pagkatuto, ipagpatuloy ang Paglalakbay

Pagkatapos mong matapos ang araling ito, bisitahin ang aming [Generative AI Learning collection](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) para patuloy mong mapalawak ang iyong kaalaman sa Generative AI!



---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama’t nagsusumikap kami para sa kawastuhan, pakatandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi eksaktong pagsasalin. Ang orihinal na dokumento sa kanyang katutubong wika ang dapat ituring na opisyal na sanggunian. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na maaaring idulot ng paggamit ng pagsasaling ito.
